In [1]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2
import tensorflow.keras as keras

time: 1.08 s (started: 2022-04-26 15:22:20 +00:00)


In [2]:
# Train - Dev - Test Generation
train_X= pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/train_X.csv')
print('loaded train_X')
dev_X = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/dev_X.csv')
print('loaded dev_X')
test1_X = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/test1_X.csv')
print('loaded test1_X')
test2_X = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/test2_X.csv')
print('loaded test2_X')
train_Y = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/train_Y.csv')
print('loaded train_Y')
dev_Y = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/dev_Y.csv')
print('loaded dev_Y')
test1_Y = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/test1_Y.csv')
print('loaded test1_Y')
test2_Y = pd.read_csv('./stock_correlation_prediction/train_dev_test/after_arima/test2_Y.csv')
print('loaded test2_Y')
train_X = train_X.loc[:, ~train_X.columns.str.contains('^Unnamed')]
dev_X = dev_X.loc[:, ~dev_X.columns.str.contains('^Unnamed')]
test1_X = test1_X.loc[:, ~test1_X.columns.str.contains('^Unnamed')]
test2_X = test2_X.loc[:, ~test2_X.columns.str.contains('^Unnamed')]
train_Y = train_Y.loc[:, ~train_Y.columns.str.contains('^Unnamed')]
dev_Y = dev_Y.loc[:, ~dev_Y.columns.str.contains('^Unnamed')]
test1_Y = test1_Y.loc[:, ~test1_Y.columns.str.contains('^Unnamed')]
test2_Y = test2_Y.loc[:, ~test2_Y.columns.str.contains('^Unnamed')]

print(train_X.shape, train_X, train_Y.shape, train_Y)

loaded train_X
loaded dev_X
loaded test1_X
loaded test2_X
loaded train_Y
loaded dev_Y
loaded test1_Y
loaded test2_Y
(55875, 20)               0         1         2         3         4         5         6  \
0      0.171160 -0.531429 -0.332825  0.068259 -0.038399 -0.213538  0.318491   
1      0.055924  0.237744 -0.941243  0.402428  0.165615  0.287995  0.313115   
2     -0.227928  0.623374 -1.120328  0.517517 -0.041095  0.440132  0.383103   
3     -0.967706  1.289477 -0.324600  0.116105  0.501316  0.073079  0.389438   
4     -1.122722  1.282431  0.089158 -0.008869  0.216753  0.174951 -0.026024   
...         ...       ...       ...       ...       ...       ...       ...   
55870 -0.560555  1.007727 -0.350772  0.282113  0.452133 -0.095661 -0.057969   
55871 -0.770758  1.302641 -0.280318  0.573215  0.505802  0.262310  0.252110   
55872 -0.635508  1.114782 -0.143117  0.583702  0.150440  0.377996  0.213962   
55873 -0.311486  0.699047 -0.020726  0.378827 -0.276535  0.296689  0.044491   
558

In [3]:
# data sampling
STEP = 20
#num_list = [STEP*i for i in range(int(1117500/STEP))]

_train_X = np.asarray(train_X).reshape((int(1117500/STEP), 20, 1))
_dev_X = np.asarray(dev_X).reshape((int(1117500/STEP), 20, 1))
_test1_X = np.asarray(test1_X).reshape((int(1117500/STEP), 20, 1))
_test2_X = np.asarray(test2_X).reshape((int(1117500/STEP), 20, 1))

_train_Y = np.asarray(train_Y).reshape(int(1117500/STEP), 1)
_dev_Y = np.asarray(dev_Y).reshape(int(1117500/STEP), 1)
_test1_Y = np.asarray(test1_Y).reshape(int(1117500/STEP), 1)
_test2_Y = np.asarray(test2_Y).reshape(int(1117500/STEP), 1)


print(_train_X.shape, _train_X, _train_Y.shape, _train_Y)

(55875, 20, 1) [[[ 0.17115954]
  [-0.53142944]
  [-0.33282545]
  ...
  [ 0.22263055]
  [-0.42584523]
  [-0.46072267]]

 [[ 0.0559241 ]
  [ 0.23774363]
  [-0.94124263]
  ...
  [-0.09089886]
  [ 0.19023268]
  [-0.39746277]]

 [[-0.22792795]
  [ 0.62337382]
  [-1.12032781]
  ...
  [-0.44696332]
  [ 0.12524743]
  [ 0.08026648]]

 ...

 [[-0.63550815]
  [ 1.11478201]
  [-0.14311722]
  ...
  [-0.35653318]
  [-0.26012937]
  [ 0.38603433]]

 [[-0.31148638]
  [ 0.6990468 ]
  [-0.02072643]
  ...
  [-0.41166356]
  [-0.01567841]
  [ 0.34875148]]

 [[ 0.19211683]
  [ 0.04050267]
  [-0.11852733]
  ...
  [-0.26786083]
  [ 0.05839987]
  [ 0.27377972]]] (55875, 1) [[ 0.01417522]
 [-0.10421273]
 [-0.24787593]
 ...
 [ 0.43692562]
 [ 0.24928944]
 [-0.19717668]]
time: 1.6 ms (started: 2022-04-26 15:22:21 +00:00)


In [4]:
#define custom activation
class Double_Tanh(Activation):
    def __init__(self, activation, **kwargs):
        super(Double_Tanh, self).__init__(activation, **kwargs)
        self.__name__ = 'double_tanh'

def double_tanh(x):
    return (K.tanh(x) * 2)

get_custom_objects().update({'double_tanh':Double_Tanh(double_tanh)})

# Model Generation
model = Sequential()
#check https://machinelearningmastery.com/use-weight-regularization-lstm-networks-time-series-forecasting/
model.add(LSTM(25, input_shape=(20,1), dropout=0.0, kernel_regularizer=l1_l2(0.00,0.00), bias_regularizer=l1_l2(0.00,0.00)))
model.add(Dense(1))
model.add(Activation(double_tanh))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])
#, kernel_regularizer=l1_l2(0,0.1), bias_regularizer=l1_l2(0,0.1),

model.summary()
print(model.metrics_names)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25)                2700      
                                                                 
 dense (Dense)               (None, 1)                 26        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 2,726
Trainable params: 2,726
Non-trainable params: 0
_________________________________________________________________
[]
time: 1.28 s (started: 2022-04-26 15:22:21 +00:00)


2022-04-26 15:22:22.717542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:22:22.720138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:22:22.720257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:22:22.720574: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
# Fitting the Model
model_scores = {}
Reg = False
d = 'hybrid_LSTM'

if Reg :
    d += '_with_reg'

epoch_num=1
for _ in range(350):

    # train the model
    dir = './stock_correlation_prediction/models/'+d
    file_list = os.listdir(dir)
    if len(file_list) != 0 :
        epoch_num = len(file_list) + 1
        recent_model_name = 'epoch'+str(epoch_num-1)+'.h5'
        filepath = './stock_correlation_prediction/models/' + d + '/' + recent_model_name
        custom_objects = {"Double_Tanh": Double_Tanh}
        with keras.utils.custom_object_scope(custom_objects):
            model = load_model(filepath)

    filepath = './stock_correlation_prediction/models/' + d + '/epoch'+str(epoch_num)+'.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, mode='min')
    callbacks_list = [checkpoint]
    if len(callbacks_list) == 0:
        model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True)
    else:
        model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True, callbacks=callbacks_list)

    # test the model
    score_train = model.evaluate(_train_X, _train_Y)
    score_dev = model.evaluate(_dev_X, _dev_Y)
    score_test1 = model.evaluate(_test1_X, _test1_Y)
    score_test2 = model.evaluate(_test2_X, _test2_Y)

    print('train set score : mse - ' + str(score_train[1]) +' / mae - ' + str(score_train[2]))
    print('dev set score : mse - ' + str(score_dev[1]) +' / mae - ' + str(score_dev[2]))
    print('test1 set score : mse - ' + str(score_test1[1]) +' / mae - ' + str(score_test1[2]))
    print('test2 set score : mse - ' + str(score_test2[1]) +' / mae - ' + str(score_test2[2]))
#.history['mean_squared_error'][0]
    # get former score data
    df = pd.read_csv("./stock_correlation_prediction/models/"+d+".csv")
    train_mse = list(df['TRAIN_MSE'])
    dev_mse = list(df['DEV_MSE'])
    test1_mse = list(df['TEST1_MSE'])
    test2_mse = list(df['TEST2_MSE'])

    train_mae = list(df['TRAIN_MAE'])
    dev_mae = list(df['DEV_MAE'])
    test1_mae = list(df['TEST1_MAE'])
    test2_mae = list(df['TEST2_MAE'])

    # append new data
    train_mse.append(score_train[1])
    dev_mse.append(score_dev[1])
    test1_mse.append(score_test1[1])
    test2_mse.append(score_test2[1])

    train_mae.append(score_train[2])
    dev_mae.append(score_dev[2])
    test1_mae.append(score_test1[2])
    test2_mae.append(score_test2[2])

    # organize newly created score dataset
    model_scores['TRAIN_MSE'] = train_mse
    model_scores['DEV_MSE'] = dev_mse
    model_scores['TEST1_MSE'] = test1_mse
    model_scores['TEST2_MSE'] = test2_mse

    model_scores['TRAIN_MAE'] = train_mae
    model_scores['DEV_MAE'] = dev_mae
    model_scores['TEST1_MAE'] = test1_mae
    model_scores['TEST2_MAE'] = test2_mae
    
    # save newly created score dataset
    model_scores_df = pd.DataFrame(model_scores)
    model_scores_df.to_csv("./stock_correlation_prediction/models/"+d+".csv")

2022-04-26 15:22:24.449505: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100


 37/112 [========>.....................] - ETA: 0s - loss: 0.1242 - mse: 0.1242 - mae: 0.2818 

2022-04-26 15:22:24.855695: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


108/112 [===========================>..] - ETA: 0s - loss: 0.0730 - mse: 0.0730 - mae: 0.2036
Epoch 00001: saving model to ./stock_correlation_prediction/models/hybrid_LSTM/epoch1.h5
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0365 - mse: 0.0365 - mae: 0.1321
train set score : mse - 0.02519172802567482 / mae - 0.10650604218244553
dev set score : mse - 0.030392378568649292 / mae - 0.1268826127052307
test1 set score : mse - 0.03561193123459816 / mae - 0.12381411343812943
test2 set score : mse - 0.03652375936508179 / mae - 0.13208788633346558
109/112 [============================>.] - ETA: 0s - loss: 0.0202 - mse: 0.0202 - mae: 0.0805
Epoch 00001: saving model to ./stock_correlation_prediction/models/hybrid_LSTM/epoch2.h5
1747/1747 [==============================] - 3s 2ms/step - loss: 0.0261 - mse: 0.0261 - mae: 0.0906
train set score : mse - 0.0188517477363348 / mae - 0.07837000489234924
dev set score : mse - 0.01919739693403244 / mae - 0.08221673220396042
test1 s

In [ ]:
filepath

In [ ]:
a = get_custom_objects()
a

In [ ]:
model.get_config()

In [ ]:
Double_Tanh(double_tanh)